# Building an Enhanced RAG System to Summarize and Converse with YouTube Videos Using LangChain, Ollama Llama 3.2, and Gradio UI
This code implements an interactive YouTube video Q&A system using a combination of tools: Gradio for the user interface, LangChain for managing the retrieval and processing of information, FAISS for efficient vector storage, and Ollama LLaMA for conversational capabilities. Here’s how it works, step by step:

## Step-by-Step Code Walkthrough
Below libraries are used in this notebook, 

- Gradio is used to build the interactive UI.
- LangChain and HuggingFaceEmbeddings provide tools for processing, embedding, and storing video data.
- FAISS is used for efficient similarity-based document retrieval.
- RecursiveCharacterTextSplitter helps in splitting the text into manageable chunks.
- ChatOllama interfaces with the LLaMA 3.2 model for conversational capabilities.
- YoutubeLoader: Extracting text from YouTube video captions.

In [8]:
import gradio as gr
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import YoutubeLoader

from langchain_ollama import ChatOllama


### 2. Processing YouTube URL
`process_youtube_url(youtube_url)` function takes a YouTube URL as input and extracts the video transcript using the YoutubeLoader.from_youtube_url method. 
It then processes the transcript by splitting it into smaller, manageable chunks using the RecursiveCharacterTextSplitter, which divides the text into segments of 512 characters. 
The resulting chunks are embedded using the HuggingFaceEmbeddings model (specifically, 'all-MiniLM-L6-v2'), which transforms the text into numerical representations suitable for machine learning tasks. These embeddings are then stored in a FAISS vector database, enabling efficient querying and retrieval of information related to the video.
Note: In order to modify the code to use captions in a different language, refer to the documentation. By default, the code reads English captions.`

In [9]:

# Global variable to hold the vector database
global_vector_db = None

# Function to process the YouTube URL and create the vector store
def process_youtube_url(youtube_url):
    loader = YoutubeLoader.from_youtube_url(youtube_url, add_video_info=False)
    documents = loader.load()

    # Initialize a text splitter
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=256,
        separators=["\n\n", "\n", " "]
    )

    # Split the documents and keep metadata
    split_docs = splitter.split_documents(documents)

    embedding_model = 'sentence-transformers/all-MiniLM-L6-v2'
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

    # Ingest into vector database
    db = FAISS.from_documents(split_docs, embeddings)
    return db


### 3. Submitting YouTube URL
The `submit_url` function processes the provided YouTube URL by calling the process_youtube_url function to create a vector store from the video's transcript. After processing, it uses a default question ("Summarize this video") to initiate a retrieval-based question-answering process. If successful, it returns a status message indicating the video has been indexed successfully, along with a summary. In case of an error, the function catches exceptions and returns an error message, ensuring a smooth user experience.

In [10]:

# Function to answer questions based on the vector store and chat history
# Submit URL function
def submit_url(youtube_url):
    global global_vector_db
    try:
        # Process the YouTube URL
        global_vector_db = process_youtube_url(youtube_url)
        
        # Default question for summarization
        default_question = "Summarize this video."
        chat_history = []  # Empty chat history for the first question
        
        # Get the summary
        summary, _ = answer_question(default_question, chat_history)
        
        # Status message
        status_message = "Video indexed successfully ✅! You can now ask questions about the video in the chatbot."
    except Exception as e:
        # Handle errors and display in the status box
        status_message = f"❌ Error processing the video: {str(e)}"
        summary = ""  # No summary in case of error
    
    # Return the status message and the summary
    return status_message, summary




### 4. Answering Questions Based on the Video
The `answer_question` function answers user questions based on the video's indexed content. It first checks if the vector database is available; if not, it prompts the user to process a video. Then, it uses the ChatOllama model and a retrieval chain to search for relevant information in the vector database and generate a concise answer. The question and corresponding answer are appended to the chat_history, allowing for a continuous, interactive conversation. If an error occurs, the function handles it by appending the error message to the chat history.

In [11]:

def answer_question(question, chat_history):
    global global_vector_db
    if global_vector_db is None:
        return "Please process a YouTube video URL first.", chat_history

    try:
        local_llm = 'llama3.2'
        llama3 = ChatOllama(model=local_llm, temperature=0)

        retriever = global_vector_db.as_retriever(search_kwargs={"k": 5})

        system_prompt = (
            "You are a video assistant tasked with answering questions based on the provided YouTube video context. "
            "Use the given context to provide accurate, concise answers in three sentences. "
            "If the context does not contain the answer, say you are not sure "
            "Context: {context}"
        )

        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{input}"),
            ]
        )

        question_answer_chain = create_stuff_documents_chain(llama3, prompt)
        chain = create_retrieval_chain(retriever, question_answer_chain)

        response = chain.invoke({"input": question})
        
        # Ensure the response contains the expected "answer" key
        if "answer" not in response:
            raise ValueError("Response does not contain an 'answer' key.")
        
        # Append the question and answer to chat history as a tuple
        chat_history.append((question, response['answer']))
        
        # Return the answer and updated chat history
        return response['answer'], chat_history
    except Exception as e:
        # Return error message in case of failure
        error_message = f"Error: {str(e)}"
        chat_history.append((question, error_message))
        return error_message, chat_history



### 5. Asking Questions via Gradio
The `ask_question` function interacts with the user to answer their query. It calls the answer_question function, passing the question and the current chat_history to retrieve an answer. The updated chat history, which includes the user's question and the assistant's response, is returned, allowing the conversation to continue smoothly. This function ensures a seamless Q&A interaction with the chatbot.

In [12]:

# Gradio interface function to ask a question
def ask_question(question, chat_history):
    response, updated_chat_history = answer_question(question, chat_history)
    return updated_chat_history, updated_chat_history  # Return the updated chat history for gr.Chatbot



### 6. Creating Gradio Interface
`create_gradio_interface()` function constructs the Gradio interface, providing a user-friendly layout for interaction with the system. It includes input fields for users to enter a YouTube URL and ask questions about the video. The interface features a submission button, status information box, and a summary box to display the video summary. Users can also ask questions, view responses, and clear the chat history as needed. Additionally, the interface includes a footer for supplementary information. This function defines the overall structure and interaction flow of the application, making it accessible and intuitive for users.

In [ ]:


def create_gradio_interface():
    with gr.Blocks() as demo:
        gr.Markdown("<h1 style='text-align: center; color: #4A90E2;'>YouTube Video Q&A</h1>")
        gr.Markdown("<p style='text-align: center;'>Enter a YouTube video URL to extract information and ask questions about it.</p>")
        
        with gr.Row():
            with gr.Column(scale=1):
                youtube_url = gr.Textbox(label="YouTube Video URL", placeholder="Enter the YouTube video URL here...", lines=1)
                submit_btn = gr.Button("Submit URL", variant="primary")
                
                # Status box for indexing completion or error messages
                status_info = gr.Textbox(
                    label="Status Info", 
                    placeholder="Indexing status will appear here...", 
                    interactive=False, 
                    lines=2
                )

                # Summary box for video summary
                summary_box = gr.Textbox(
                    label="Video Summary", 
                    placeholder="Summary will appear here...", 
                    interactive=False, 
                    lines=6
                )
                
                # Link submit button to the submit_url function
                submit_btn.click(fn=submit_url, inputs=youtube_url, outputs=[status_info, summary_box])

            with gr.Column(scale=1):
                chat_history = gr.Chatbot()
                question = gr.Textbox(label="Your Question", placeholder="Ask a question about the video...", lines=1)
                ask_btn = gr.Button("Ask Question", variant="primary")
                clear_btn = gr.Button("Clear Chat", variant="secondary")
                state = gr.State([])

                # Link ask button to the ask_question function
                ask_btn.click(fn=ask_question, inputs=[question, state], outputs=[chat_history, state])
                
                # Clear chat history
                clear_btn.click(fn=lambda: ([], []), inputs=[], outputs=[chat_history, state])  # Clear chat history

        # Add a footer or additional information
        gr.Markdown("<footer style='text-align: center; margin-top: 20px;'>© Vikram Bhat</footer>")

    return demo



### 7. Launching the Gradio Interface
`interface.launch` starts the Gradio interface, allowing users to interact with the system by entering a YouTube URL, asking questions, and receiving answers.

In [ ]:
# Launch the Gradio interface
interface = create_gradio_interface()
interface.launch(share=True)


Enter the enhanced Retrieval-Augmented Generation (RAG) system. By integrating cutting-edge tools like LangChain, Ollama Llama 3.2, and Gradio UI, this application takes video summarization and Q&A to the next level. 
The RAG system is designed to not only summarize YouTube videos using the captions but also enable interactive conversations with the content, allowing users to ask questions and receive precise, contextually relevant answers in real-time.